In [1]:
# Conncect to google drive to access dataset.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# unzip images obtained from SCFace dataset. It cannot be downloaded online due to the legal agreement.
!unzip '/content/drive/MyDrive/cropped_humans.zip'

Archive:  /content/drive/MyDrive/cropped_humans.zip
  inflating: cropped_humans/001_frontal.jpg  
  inflating: cropped_humans/001_L1.jpg  
  inflating: cropped_humans/001_L2.jpg  
  inflating: cropped_humans/001_L3.jpg  
  inflating: cropped_humans/001_L4.jpg  
  inflating: cropped_humans/001_R1.jpg  
  inflating: cropped_humans/001_R2.jpg  
  inflating: cropped_humans/001_R3.jpg  
  inflating: cropped_humans/001_R4.jpg  
  inflating: cropped_humans/002_frontal.jpg  
  inflating: cropped_humans/002_L1.jpg  
  inflating: cropped_humans/002_L2.jpg  
  inflating: cropped_humans/002_L3.jpg  
  inflating: cropped_humans/002_L4.jpg  
  inflating: cropped_humans/002_R1.jpg  
  inflating: cropped_humans/002_R2.jpg  
  inflating: cropped_humans/002_R3.jpg  
  inflating: cropped_humans/002_R4.jpg  
  inflating: cropped_humans/003_frontal.jpg  
  inflating: cropped_humans/003_L1.jpg  
  inflating: cropped_humans/003_L2.jpg  
  inflating: cropped_humans/003_L3.jpg  
  inflating: cropped_humans/003

In [3]:
# This zip file contains 4 different types of face masks with front, left and right angel.
# It will be used to impose masks on SCFace human face dataset
!unzip '/content/drive/MyDrive/MSProjectData/Dataset3/Masks.zip' 

Archive:  /content/drive/MyDrive/MSProjectData/Dataset3/Masks.zip
   creating: Masks/Black/
  inflating: Masks/Black/Front_Black.png  
  inflating: Masks/Black/Left_Black.png  
  inflating: Masks/Black/Right_Black.png  
   creating: Masks/Cloth/
 extracting: Masks/Cloth/Front_Cloth.png  
 extracting: Masks/Cloth/Left_Cloth.png  
 extracting: Masks/Cloth/Right_Cloth.png  
   creating: Masks/Medical/
  inflating: Masks/Medical/Front_Medical.png  
  inflating: Masks/Medical/Left_Medical.png  
  inflating: Masks/Medical/Right_Medical.png  
   creating: Masks/N95/
  inflating: Masks/N95/Front_N95.png  
  inflating: Masks/N95/Left_N95.png  
  inflating: Masks/N95/Right_N95.png  


In [4]:
!pip install mtcnn
from mtcnn import MTCNN
detector=MTCNN()

     |████████████████████████████████| 2.3MB 13.9MB/s 


In [5]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import pickle
from sklearn.model_selection import KFold

In [6]:
# Data is stored in folders. Every folder contains several images of a single subject with different angels.
# There are total 130 subjects
folder_address="/content/cropped_humans/"
file_names=os.listdir(folder_address)
useful_files=[]
for myfile in file_names:
  if "frontal" in myfile or "L1" in myfile or "R1" in myfile: 
    useful_files.append(myfile)
useful_files.sort()

workfiles=[]
for i in range(1,len(useful_files)+1):
  front_file=f'{i:03}'+"_"+"frontal.jpg"
  left_file=f'{i:03}'+"_"+"L1.jpg"
  right_file=f'{i:03}'+"_"+"R1.jpg"
  # only keep a subject that has front, left and right pose available.
  if front_file in useful_files and left_file in useful_files and right_file in useful_files: 
    workfiles.append(front_file)
    workfiles.append(left_file)
    workfiles.append(right_file)
workfiles.sort()

In [7]:
# Let's only keep subjects whose front,left and right angle poses can be detected by MTCNN
i=0
mtcnn_approvedfiles=[]
count=0
while i<len(workfiles):
  left_file=cv2.imread(folder_address+workfiles[i])
  left_file=cv2.resize(left_file,(128,128))
  left_file_RGB=cv2.cvtColor(left_file,cv2.COLOR_BGR2RGB)

  right_file=cv2.imread(folder_address+workfiles[i+1])
  right_file=cv2.resize(right_file,(128,128))
  right_file_RGB=cv2.cvtColor(right_file,cv2.COLOR_BGR2RGB)

  front_file=cv2.imread(folder_address+workfiles[i+2])
  front_file=cv2.resize(front_file,(128,128))
  front_file_RGB=cv2.cvtColor(front_file,cv2.COLOR_BGR2RGB)


  left_detection=detector.detect_faces(left_file_RGB)
  right_detection=detector.detect_faces(right_file_RGB)
  front_detection=detector.detect_faces(front_file_RGB)

  if left_detection!=[] and right_detection!=[] and front_detection!=[]:
    mtcnn_approvedfiles.append(workfiles[i])
    mtcnn_approvedfiles.append(workfiles[i+1])
    mtcnn_approvedfiles.append(workfiles[i+2])
  i=i+3

In [8]:
print("MTCNN has approved total",len(mtcnn_approvedfiles),"faces/files of total",int(len(mtcnn_approvedfiles)/3),"subjects")

MTCNN has approved total 237 faces/files of total 79 subjects


In [9]:
def GenerateData(decideFileAngle,decideMaskangle,adjustYMin): 
  masked_data=[]
  mtcnn_approvedfiles_length=int(len(mtcnn_approvedfiles)/3)
  for count in range(1,131):
    myfile=f'{count:03}'+"_"+decideFileAngle
    if decideFileAngle in myfile and myfile in mtcnn_approvedfiles:
      file_name=folder_address+myfile
      img=cv2.imread(file_name)
      img=cv2.resize(img,(128,128))
      img_bg=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

      coordinates=detector.detect_faces(img_bg)
      if coordinates==[]:
        masked_data[folder]=np.zeros((128,128,3))
        continue
      
      # Start Y-Max Calculation
      def findym(p1,p2,para,nose_height):
        if (p1-p2)*para<=(128-nose_height) and para<2.8:
          result=findym(p1,p2,para+0.1,nose_height)
        else:
          result=(p1-p2)*para
        return result

      
      p1=coordinates[0]['keypoints']['mouth_left'][1]
      p2=coordinates[0]['keypoints']['nose'][1]
      para=1
      nose_height=coordinates[0]['keypoints']['nose'][1]
      adjustment=int(findym(p1,p2,para,nose_height))

      y_max=coordinates[0]['keypoints']['nose'][1]+adjustment

      # End Y-Max Calculation

      if adjustYMin=="Face_Covered":
        y_min=coordinates[0]['keypoints']['left_eye'][1]+1
      elif adjustYMin=="Face_2Covered":
        y_min=coordinates[0]['keypoints']['left_eye'][1]+2
      elif adjustYMin=="Face_3Covered":
        y_min=coordinates[0]['keypoints']['left_eye'][1]+3
      elif adjustYMin=="Nose_Uncovered":
        y_min=coordinates[0]['keypoints']['nose'][1]+4
      elif adjustYMin=="NoseMouth_Uncovered":
        y_min=coordinates[0]['keypoints']['mouth_left'][1]
      elif adjustYMin=="Chin_Uncovered":
        y_min=coordinates[0]['keypoints']['left_eye'][1]+1
        y_max=coordinates[0]['keypoints']['mouth_left'][1]
      else:
        continue

      x_max=coordinates[0]['keypoints']['nose'][0]
      x_min=coordinates[0]['keypoints']['nose'][0]


      x_max=coordinates[0]['keypoints']['right_eye'][0]+int((coordinates[0]['keypoints']['right_eye'][0]-coordinates[0]['keypoints']['left_eye'][0])/2)
      x_min=coordinates[0]['keypoints']['left_eye'][0]-int((coordinates[0]['keypoints']['right_eye'][0]-coordinates[0]['keypoints']['left_eye'][0])/2)
      if x_min<=0:
        x_min=0
      if x_max>=128:
        x_max=128
      if y_max>=128:
        y_max=128

      mask=cv2.imread(decideMaskangle)
      mask=cv2.resize(mask,(x_max-x_min,y_max-y_min))



      i=y_min
      while i<y_max:
        j=x_min
        while j<x_max:
          if mask[i-y_min][j-x_min][0]!=0 and mask[i-y_min][j-x_min][1]!=0 and mask[i-y_min][j-x_min][2]!=0:
            img[i][j][0]=mask[i-y_min][j-x_min][0]
            img[i][j][1]=mask[i-y_min][j-x_min][1]
            img[i][j][2]=mask[i-y_min][j-x_min][2]
          j=j+1
        i=i+1
      img=cv2.resize(img,(16,16))
      masked_data.append(img)
  return masked_data

In [10]:
################################################### Let's put N95 Masks on people's faces

front_mask="/content/Masks/N95/Front_N95.png"
left_mask="/content/Masks/N95/Left_N95.png"
right_mask="/content/Masks/N95/Right_N95.png"

## Generate Correct Mask
Correct_Front=GenerateData('frontal.jpg',front_mask,'Face_Covered')
Correct_FrontSecond=GenerateData('frontal.jpg',front_mask,'Face_2Covered')
Correct_FrontThird=GenerateData('frontal.jpg',front_mask,'Face_3Covered')

Correct_Left=GenerateData('L1.jpg',left_mask,'Face_Covered')
Correct_LeftSecond=GenerateData('L1.jpg',left_mask,'Face_2Covered')
Correct_LeftThird=GenerateData('L1.jpg',left_mask,'Face_3Covered')

Correct_Right=GenerateData('R1.jpg',right_mask,'Face_Covered')
Correct_RightSecond=GenerateData('R1.jpg',right_mask,'Face_2Covered')
Correct_RightThird=GenerateData('R1.jpg',right_mask,'Face_3Covered')

# Generate Incorrect Mask
Incorrect_FrontNoseUncovered=GenerateData('frontal.jpg',front_mask,'Nose_Uncovered')
Incorrect_FrontNoseMouthUncovered=GenerateData('frontal.jpg',front_mask,'NoseMouth_Uncovered')
Incorrect_FrontChinUncovered=GenerateData('frontal.jpg',front_mask,'Chin_Uncovered')

Incorrect_LeftNoseUncovered=GenerateData('L1.jpg',left_mask,'Nose_Uncovered')
Incorrect_LeftNoseMouthUncovered=GenerateData('L1.jpg',left_mask,'NoseMouth_Uncovered')
Incorrect_LeftChinUncovered=GenerateData('L1.jpg',left_mask,'Chin_Uncovered')

Incorrect_RightNoseUncovered=GenerateData('R1.jpg',right_mask,'Nose_Uncovered')
Incorrect_RightNoseMouthUncovered=GenerateData('R1.jpg',right_mask,'NoseMouth_Uncovered')
Incorrect_RightChinUncovered=GenerateData('R1.jpg',right_mask,'Chin_Uncovered')

CorrectN95=[]
for i in range(len(Correct_Front)):
  folder=[]

  folder.append(Correct_Front[i])
  folder.append(Correct_FrontSecond[i])
  folder.append(Correct_FrontThird[i])

  folder.append(Correct_Left[i])
  folder.append(Correct_LeftSecond[i])
  folder.append(Correct_LeftThird[i])

  folder.append(Correct_Right[i])
  folder.append(Correct_RightSecond[i])
  folder.append(Correct_RightThird[i])

  CorrectN95.append(folder)
CorrectN95=np.asarray(CorrectN95)

IncorrectN95=[]
for i in range(len(Incorrect_FrontNoseUncovered)):
  folder=[]

  folder.append(Incorrect_FrontNoseUncovered[i])
  folder.append(Incorrect_FrontNoseMouthUncovered[i])
  folder.append(Incorrect_FrontChinUncovered[i])

  folder.append(Incorrect_LeftNoseUncovered[i])
  folder.append(Incorrect_LeftNoseMouthUncovered[i])
  folder.append(Incorrect_LeftChinUncovered[i])

  folder.append(Incorrect_RightNoseUncovered[i])
  folder.append(Incorrect_RightNoseMouthUncovered[i])
  folder.append(Incorrect_RightChinUncovered[i])
  
  IncorrectN95.append(folder)
IncorrectN95=np.asarray(IncorrectN95)

print(CorrectN95.shape,IncorrectN95.shape)

################################################### End of putting N95 mask on people's faces

(79, 9, 16, 16, 3) (79, 9, 16, 16, 3)


In [11]:
################################################### Let's put Cloth Masks on people's faces

front_mask="/content/Masks/Cloth/Front_Cloth.png"
left_mask="/content/Masks/Cloth/Left_Cloth.png"
right_mask="/content/Masks/Cloth/Right_Cloth.png"


# Generate Correct Mask
Correct_Front=GenerateData('frontal.jpg',front_mask,'Face_Covered')
Correct_FrontSecond=GenerateData('frontal.jpg',front_mask,'Face_2Covered')
Correct_FrontThird=GenerateData('frontal.jpg',front_mask,'Face_3Covered')

Correct_Left=GenerateData('L1.jpg',left_mask,'Face_Covered')
Correct_LeftSecond=GenerateData('L1.jpg',left_mask,'Face_2Covered')
Correct_LeftThird=GenerateData('L1.jpg',left_mask,'Face_3Covered')

Correct_Right=GenerateData('R1.jpg',right_mask,'Face_Covered')
Correct_RightSecond=GenerateData('R1.jpg',right_mask,'Face_2Covered')
Correct_RightThird=GenerateData('R1.jpg',right_mask,'Face_3Covered')

# Generate Incorrect Mask
Incorrect_FrontNoseUncovered=GenerateData('frontal.jpg',front_mask,'Nose_Uncovered')
Incorrect_FrontNoseMouthUncovered=GenerateData('frontal.jpg',front_mask,'NoseMouth_Uncovered')
Incorrect_FrontChinUncovered=GenerateData('frontal.jpg',front_mask,'Chin_Uncovered')

Incorrect_LeftNoseUncovered=GenerateData('L1.jpg',left_mask,'Nose_Uncovered')
Incorrect_LeftNoseMouthUncovered=GenerateData('L1.jpg',left_mask,'NoseMouth_Uncovered')
Incorrect_LeftChinUncovered=GenerateData('L1.jpg',left_mask,'Chin_Uncovered')

Incorrect_RightNoseUncovered=GenerateData('R1.jpg',right_mask,'Nose_Uncovered')
Incorrect_RightNoseMouthUncovered=GenerateData('R1.jpg',right_mask,'NoseMouth_Uncovered')
Incorrect_RightChinUncovered=GenerateData('R1.jpg',right_mask,'Chin_Uncovered')

CorrectCloth=[]
for i in range(len(Correct_Front)):
  folder=[]
  folder.append(Correct_Front[i])
  folder.append(Correct_FrontSecond[i])
  folder.append(Correct_FrontThird[i])

  folder.append(Correct_Left[i])
  folder.append(Correct_LeftSecond[i])
  folder.append(Correct_LeftThird[i])

  folder.append(Correct_Right[i])
  folder.append(Correct_RightSecond[i])
  folder.append(Correct_RightThird[i])
  CorrectCloth.append(folder)
CorrectCloth=np.asarray(CorrectCloth)

IncorrectCloth=[]
for i in range(len(Incorrect_FrontNoseUncovered)):
  folder=[]
  folder.append(Incorrect_FrontNoseUncovered[i])
  folder.append(Incorrect_FrontNoseMouthUncovered[i])
  folder.append(Incorrect_FrontChinUncovered[i])

  folder.append(Incorrect_LeftNoseUncovered[i])
  folder.append(Incorrect_LeftNoseMouthUncovered[i])
  folder.append(Incorrect_LeftChinUncovered[i])

  folder.append(Incorrect_RightNoseUncovered[i])
  folder.append(Incorrect_RightNoseMouthUncovered[i])
  folder.append(Incorrect_RightChinUncovered[i])
  IncorrectCloth.append(folder)
IncorrectCloth=np.asarray(IncorrectCloth)

print(CorrectCloth.shape,IncorrectCloth.shape)

################################################### End of putting Cloth masks on people's faces

(79, 9, 16, 16, 3) (79, 9, 16, 16, 3)


In [12]:
################################################### Let's put Medical Masks on people's faces

front_mask="/content/Masks/Medical/Front_Medical.png"
left_mask="/content/Masks/Medical/Left_Medical.png"
right_mask="/content/Masks/Medical/Right_Medical.png"


# Generate Correct Mask
Correct_Front=GenerateData('frontal.jpg',front_mask,'Face_Covered')
Correct_FrontSecond=GenerateData('frontal.jpg',front_mask,'Face_2Covered')
Correct_FrontThird=GenerateData('frontal.jpg',front_mask,'Face_3Covered')

Correct_Left=GenerateData('L1.jpg',left_mask,'Face_Covered')
Correct_LeftSecond=GenerateData('L1.jpg',left_mask,'Face_2Covered')
Correct_LeftThird=GenerateData('L1.jpg',left_mask,'Face_3Covered')

Correct_Right=GenerateData('R1.jpg',right_mask,'Face_Covered')
Correct_RightSecond=GenerateData('R1.jpg',right_mask,'Face_2Covered')
Correct_RightThird=GenerateData('R1.jpg',right_mask,'Face_3Covered')

# Generate Incorrect Mask
Incorrect_FrontNoseUncovered=GenerateData('frontal.jpg',front_mask,'Nose_Uncovered')
Incorrect_FrontNoseMouthUncovered=GenerateData('frontal.jpg',front_mask,'NoseMouth_Uncovered')
Incorrect_FrontChinUncovered=GenerateData('frontal.jpg',front_mask,'Chin_Uncovered')

Incorrect_LeftNoseUncovered=GenerateData('L1.jpg',left_mask,'Nose_Uncovered')
Incorrect_LeftNoseMouthUncovered=GenerateData('L1.jpg',left_mask,'NoseMouth_Uncovered')
Incorrect_LeftChinUncovered=GenerateData('L1.jpg',left_mask,'Chin_Uncovered')

Incorrect_RightNoseUncovered=GenerateData('R1.jpg',right_mask,'Nose_Uncovered')
Incorrect_RightNoseMouthUncovered=GenerateData('R1.jpg',right_mask,'NoseMouth_Uncovered')
Incorrect_RightChinUncovered=GenerateData('R1.jpg',right_mask,'Chin_Uncovered')
CorrectMedical=[]
for i in range(len(Correct_Front)):
  folder=[]
  folder.append(Correct_Front[i])
  folder.append(Correct_FrontSecond[i])
  folder.append(Correct_FrontThird[i])

  folder.append(Correct_Left[i])
  folder.append(Correct_LeftSecond[i])
  folder.append(Correct_LeftThird[i])

  folder.append(Correct_Right[i])
  folder.append(Correct_RightSecond[i])
  folder.append(Correct_RightThird[i])
  CorrectMedical.append(folder)
CorrectMedical=np.asarray(CorrectMedical)

IncorrectMedical=[]
for i in range(len(Incorrect_FrontNoseUncovered)):
  folder=[]
  folder.append(Incorrect_FrontNoseUncovered[i])
  folder.append(Incorrect_FrontNoseMouthUncovered[i])
  folder.append(Incorrect_FrontChinUncovered[i])

  folder.append(Incorrect_LeftNoseUncovered[i])
  folder.append(Incorrect_LeftNoseMouthUncovered[i])
  folder.append(Incorrect_LeftChinUncovered[i])

  folder.append(Incorrect_RightNoseUncovered[i])
  folder.append(Incorrect_RightNoseMouthUncovered[i])
  folder.append(Incorrect_RightChinUncovered[i])
  IncorrectMedical.append(folder)
IncorrectMedical=np.asarray(IncorrectMedical)

print(CorrectMedical.shape,IncorrectMedical.shape)

################################################### End of putting Medical masks on people's faces

(79, 9, 16, 16, 3) (79, 9, 16, 16, 3)


In [13]:
################################################### Let's put Black Cloth Masks on people's faces

front_mask="/content/Masks/Black/Front_Black.png"
left_mask="/content/Masks/Black/Left_Black.png"
right_mask="/content/Masks/Black/Right_Black.png"


# Generate Correct Mask
Correct_Front=GenerateData('frontal.jpg',front_mask,'Face_Covered')
Correct_FrontSecond=GenerateData('frontal.jpg',front_mask,'Face_2Covered')
Correct_FrontThird=GenerateData('frontal.jpg',front_mask,'Face_3Covered')

Correct_Left=GenerateData('L1.jpg',left_mask,'Face_Covered')
Correct_LeftSecond=GenerateData('L1.jpg',left_mask,'Face_2Covered')
Correct_LeftThird=GenerateData('L1.jpg',left_mask,'Face_3Covered')

Correct_Right=GenerateData('R1.jpg',right_mask,'Face_Covered')
Correct_RightSecond=GenerateData('R1.jpg',right_mask,'Face_2Covered')
Correct_RightThird=GenerateData('R1.jpg',right_mask,'Face_3Covered')

# Generate Incorrect Mask
Incorrect_FrontNoseUncovered=GenerateData('frontal.jpg',front_mask,'Nose_Uncovered')
Incorrect_FrontNoseMouthUncovered=GenerateData('frontal.jpg',front_mask,'NoseMouth_Uncovered')
Incorrect_FrontChinUncovered=GenerateData('frontal.jpg',front_mask,'Chin_Uncovered')

Incorrect_LeftNoseUncovered=GenerateData('L1.jpg',left_mask,'Nose_Uncovered')
Incorrect_LeftNoseMouthUncovered=GenerateData('L1.jpg',left_mask,'NoseMouth_Uncovered')
Incorrect_LeftChinUncovered=GenerateData('L1.jpg',left_mask,'Chin_Uncovered')

Incorrect_RightNoseUncovered=GenerateData('R1.jpg',right_mask,'Nose_Uncovered')
Incorrect_RightNoseMouthUncovered=GenerateData('R1.jpg',right_mask,'NoseMouth_Uncovered')
Incorrect_RightChinUncovered=GenerateData('R1.jpg',right_mask,'Chin_Uncovered')

CorrectBlack=[]
for i in range(len(Correct_Front)):
  folder=[]
  folder.append(Correct_Front[i])
  folder.append(Correct_FrontSecond[i])
  folder.append(Correct_FrontThird[i])

  folder.append(Correct_Left[i])
  folder.append(Correct_LeftSecond[i])
  folder.append(Correct_LeftThird[i])

  folder.append(Correct_Right[i])
  folder.append(Correct_RightSecond[i])
  folder.append(Correct_RightThird[i])
  CorrectBlack.append(folder)
CorrectBlack=np.asarray(CorrectBlack)

IncorrectBlack=[]
for i in range(len(Incorrect_FrontNoseUncovered)):
  folder=[]
  folder.append(Incorrect_FrontNoseUncovered[i])
  folder.append(Incorrect_FrontNoseMouthUncovered[i])
  folder.append(Incorrect_FrontChinUncovered[i])

  folder.append(Incorrect_LeftNoseUncovered[i])
  folder.append(Incorrect_LeftNoseMouthUncovered[i])
  folder.append(Incorrect_LeftChinUncovered[i])

  folder.append(Incorrect_RightNoseUncovered[i])
  folder.append(Incorrect_RightNoseMouthUncovered[i])
  folder.append(Incorrect_RightChinUncovered[i])
  IncorrectBlack.append(folder)
IncorrectBlack=np.asarray(IncorrectBlack)

print(CorrectBlack.shape,IncorrectBlack.shape)

################################################### End of putting Black Cloth Mask on people's faces

(79, 9, 16, 16, 3) (79, 9, 16, 16, 3)


In [14]:
####################################################Let's include the original data in incorrect features
original_incorrect=[]
i=0
while i<len(mtcnn_approvedfiles):
  store_subject=[]
  img=cv2.imread(folder_address+mtcnn_approvedfiles[i])
  store_subject.append(cv2.resize(img,(16,16)))
  img=cv2.imread(folder_address+mtcnn_approvedfiles[i+1])
  store_subject.append(cv2.resize(img,(16,16)))
  img=cv2.imread(folder_address+mtcnn_approvedfiles[i+2])
  store_subject.append(cv2.resize(img,(16,16)))
  original_incorrect.append(store_subject)
  i=i+3
original_incorrect=np.asarray(original_incorrect)
print(original_incorrect.shape)
####################################################End of Including original data in correct features

(79, 3, 16, 16, 3)


In [15]:
#######Lets merge data generated by differnet types of masks
correct_data=np.concatenate((CorrectN95,CorrectCloth,CorrectMedical,CorrectBlack),axis=1)
incorrect_data=np.concatenate((IncorrectN95,IncorrectCloth,IncorrectMedical,IncorrectBlack,original_incorrect),axis=1)

In [16]:
print("Correctly Masked Data Created Shape",correct_data.shape,"\nIncorrectly Masked Data Created Shape",incorrect_data.shape)

Correctly Masked Data Created Shape (79, 36, 16, 16, 3) 
Incorrectly Masked Data Created Shape (79, 39, 16, 16, 3)


In [17]:
# Let's convert correct and incorrect data into grayscale
correct_data=np.dot(correct_data[...,:3], [0.299, 0.587, 0.144])
incorrect_data=np.dot(incorrect_data[...,:3], [0.299, 0.587, 0.144])
print("Correctly Masked Data Created Shape",correct_data.shape,"\nIncorrectly Masked Data Created Shape",incorrect_data.shape)

Correctly Masked Data Created Shape (79, 36, 16, 16) 
Incorrectly Masked Data Created Shape (79, 39, 16, 16)


In [25]:
# Let's reshape correct and incorrect data
correct_data=np.reshape(correct_data,(correct_data.shape[0],correct_data.shape[1],correct_data.shape[2],correct_data.shape[3],1))
incorrect_data=np.reshape(incorrect_data,(incorrect_data.shape[0],incorrect_data.shape[1],incorrect_data.shape[2],correct_data.shape[3],1))
print("Correctly Masked Data Created Shape",correct_data.shape,"\nIncorrectly Masked Data Created Shape",incorrect_data.shape)

Correctly Masked Data Created Shape (79, 36, 16, 16, 1) 
Incorrectly Masked Data Created Shape (79, 39, 16, 16, 1)


In [26]:
# Save correctly and incorrectly masked features
from numpy import savez_compressed
savez_compressed('/content/drive/MyDrive/MSProjectData/Dataset3/Correct/correct_features.npz', correct_data)
savez_compressed('/content/drive/MyDrive/MSProjectData/Dataset3/Incorrect/incorrect_features.npz', incorrect_data)